In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tensorboardX import SummaryWriter

plt.ion()   # interactive mode
%matplotlib inline

Load Data
---------


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [3]:
def train_model(model, criterion, optimizer, scheduler, writer=None, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    if isinstance(outputs, tuple):
                        outputs=outputs[0]
                    
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            writer.add_scalar(phase+"/loss", epoch_loss, epoch)
            writer.add_scalar(phase+"/acc", epoch_acc, epoch)


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Visualizing the model predictions

In [4]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [5]:
model_ft = models.inception_v3(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)

writer = SummaryWriter('./logs/inception_full')

### Train and evaluate



In [6]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, writer,
                       num_epochs=40)

Epoch 0/39
----------
train Loss: 1.2343 Acc: 0.5450
val Loss: 0.5817 Acc: 0.7935

Epoch 1/39
----------
train Loss: 0.6461 Acc: 0.7807
val Loss: 0.3549 Acc: 0.8947

Epoch 2/39
----------
train Loss: 0.4236 Acc: 0.8581
val Loss: 0.2727 Acc: 0.9028

Epoch 3/39
----------
train Loss: 0.2893 Acc: 0.9056
val Loss: 0.2477 Acc: 0.9069

Epoch 4/39
----------
train Loss: 0.2188 Acc: 0.9260
val Loss: 0.2002 Acc: 0.9433

Epoch 5/39
----------
train Loss: 0.1774 Acc: 0.9450
val Loss: 0.1601 Acc: 0.9474

Epoch 6/39
----------
train Loss: 0.0797 Acc: 0.9759
val Loss: 0.1091 Acc: 0.9676

Epoch 7/39
----------
train Loss: 0.0595 Acc: 0.9837
val Loss: 0.1215 Acc: 0.9676

Epoch 8/39
----------
train Loss: 0.0541 Acc: 0.9843
val Loss: 0.1004 Acc: 0.9717

Epoch 9/39
----------
train Loss: 0.0459 Acc: 0.9871
val Loss: 0.0987 Acc: 0.9676

Epoch 10/39
----------
train Loss: 0.0334 Acc: 0.9927
val Loss: 0.1175 Acc: 0.9676

Epoch 11/39
----------
train Loss: 0.0313 Acc: 0.9912
val Loss: 0.1082 Acc: 0.9676

Ep

## Freeze first few layers

In [12]:
model_conv = torchvision.models.inception_v3(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
    
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, child in model_conv.named_children():
    if "Conv2d_4a_3x3" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)
    
# To view which layers are freeze and which layers are not freezed:
for name, child in model_conv.named_children():
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)


optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 6 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=6, gamma=0.1)

writer = SummaryWriter('./logs/inception_freezed_first4')

conv.weight False
bn.weight False
bn.bias False
conv.weight False
bn.weight False
bn.bias False
conv.weight False
bn.weight False
bn.bias False
conv.weight False
bn.weight False
bn.bias False
conv.weight False
bn.weight False
bn.bias False
branch1x1.conv.weight True
branch1x1.bn.weight True
branch1x1.bn.bias True
branch5x5_1.conv.weight True
branch5x5_1.bn.weight True
branch5x5_1.bn.bias True
branch5x5_2.conv.weight True
branch5x5_2.bn.weight True
branch5x5_2.bn.bias True
branch3x3dbl_1.conv.weight True
branch3x3dbl_1.bn.weight True
branch3x3dbl_1.bn.bias True
branch3x3dbl_2.conv.weight True
branch3x3dbl_2.bn.weight True
branch3x3dbl_2.bn.bias True
branch3x3dbl_3.conv.weight True
branch3x3dbl_3.bn.weight True
branch3x3dbl_3.bn.bias True
branch_pool.conv.weight True
branch_pool.bn.weight True
branch_pool.bn.bias True
branch1x1.conv.weight True
branch1x1.bn.weight True
branch1x1.bn.bias True
branch5x5_1.conv.weight True
branch5x5_1.bn.weight True
branch5x5_1.bn.bias True
branch5x5_2.conv

In [13]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, writer, num_epochs=50)

Epoch 0/49
----------
train Loss: 1.1482 Acc: 0.5653
val Loss: 0.7919 Acc: 0.7490

Epoch 1/49
----------
train Loss: 0.5498 Acc: 0.8155
val Loss: 0.4549 Acc: 0.8381

Epoch 2/49
----------
train Loss: 0.3617 Acc: 0.8807
val Loss: 0.2834 Acc: 0.9150

Epoch 3/49
----------
train Loss: 0.2574 Acc: 0.9174
val Loss: 0.2528 Acc: 0.9190

Epoch 4/49
----------
train Loss: 0.1898 Acc: 0.9385
val Loss: 0.2108 Acc: 0.9150

Epoch 5/49
----------
train Loss: 0.1426 Acc: 0.9568
val Loss: 0.2350 Acc: 0.9312

Epoch 6/49
----------
train Loss: 0.0763 Acc: 0.9789
val Loss: 0.1560 Acc: 0.9514

Epoch 7/49
----------
train Loss: 0.0539 Acc: 0.9875
val Loss: 0.1657 Acc: 0.9514

Epoch 8/49
----------
train Loss: 0.0462 Acc: 0.9877
val Loss: 0.1712 Acc: 0.9474

Epoch 9/49
----------
train Loss: 0.0348 Acc: 0.9929
val Loss: 0.1785 Acc: 0.9514

Epoch 10/49
----------
train Loss: 0.0352 Acc: 0.9910
val Loss: 0.1609 Acc: 0.9514

Epoch 11/49
----------
train Loss: 0.0272 Acc: 0.9942
val Loss: 0.1744 Acc: 0.9555

Ep